# Tucson Monthly Reports
- https://jira.abcmouse.com/browse/PDA-6867
- https://jira.abcmouse.com/browse/PDA-7082

The purpose of this report is to provide monthly usage data for their Native American Student population. The first ticket outlines that they need usage (minutes played), while the second determines that they need skills mastered. 

The cadence of this report is to provide data for each month, from July 2023 until May 2024. 

In [1]:
# Query to get the data 
query = """
with base as (
   -- Get rostering information based on what is active on the last day of the month
SELECT
    AOFL_PRODUCT,
    STUDENT_ID,
    ORGANIZATION_ID,
    ORGANIZATION_NAME,
    MONTH as final_month,
    DATE as final_day,
    -- Concatenate together values from different active grades, schools, classrooms, and teachers
    LISTAGG(DISTINCT GRADE, '; ') WITHIN GROUP (ORDER BY GRADE ASC) AS GRADE, 
    LISTAGG(DISTINCT SCHOOL_NAME, '; ') WITHIN GROUP (ORDER BY SCHOOL_NAME ASC) AS SCHOOL_NAME, 
    LISTAGG(DISTINCT CLASSROOM_NAME, '; ') WITHIN GROUP (ORDER BY CLASSROOM_NAME ASC) AS CLASSROOM_NAME,
    LISTAGG(DISTINCT TEACHER_ID, '; ') WITHIN GROUP (ORDER BY TEACHER_ID ASC) AS teachers
FROM SCHOOLS_DW.DM.MASTERY_LICENSED_ACTIVE_DAILY_SNAPSHOT
WHERE ORGANIZATION_NAME = 'Tucson Unified School District'
  -- AND DATE = DATEADD(day, -1, DATE_TRUNC('month', CURRENT_DATE())) // Defined in the scope of the project
  AND DATE = DATEADD(day, -1, CURRENT_DATE()) // Defined in the scope of the project
  AND IS_LICENSED_SCHOOL = TRUE
  AND IS_LICENSED_CLASSROOM = TRUE
  AND STUDENT_GROUP_LINK_STATUS = 'active'
  AND IS_LICENSED_STUDENT = TRUE
  AND (DATE = LAST_DAY(DATE) OR DATE = DATEADD(day, -1, CURRENT_DATE()))-- Retrieves the last day of the month for the given DATE
GROUP BY 1, 2, 3, 4, 5, 6
), distinct_daily_usage as (
// Since every line has multiple rows for a given day based on rostering information
// I need to deduplicate the usage before aggregating. 
SELECT DISTINCT
    AOFL_PRODUCT,
    STUDENT_ID,
    DATE,
    WEEK,
    MONTH,
    time_spend,
    HAS_PLACEMENT
FROM SCHOOLS_DW.DM.MASTERY_LICENSED_ACTIVE_DAILY_SNAPSHOT 
WHERE ORGANIZATION_NAME = 'Tucson Unified School District'
-- AND DATE BETWEEN '2023-07-01' AND DATEADD(day, -1, DATE_TRUNC('month', CURRENT_DATE())) // Defined in the scope of the project
AND DATE BETWEEN '2023-07-01' AND DATEADD(day, -1, CURRENT_DATE()) // Yesterday for QA
), monthly_usage as (
 SELECT 
    AOFL_PRODUCT,
    STUDENT_ID,
    MONTH,
    sum(time_spend) / 60 as monthly_usage_minutes,
    MAX(HAS_PLACEMENT) as HAS_PLACEMENT,
    MAX(iff(time_spend > 0, DATE,null)) as max_login_date,
    MIN(iff(time_spend > 0, DATE,null)) as MIN_login_date,
    ARRAY_DISTINCT(ARRAY_AGG(DISTINCT IFF(time_spend > 0, WEEK, NULL))) as weeks_aggregated
FROM distinct_daily_usage
GROUP BY 1,2,3
), total_usage as (
SELECT 
    AOFL_PRODUCT,
    STUDENT_ID,
    NULLIF(sum(monthly_usage_minutes), 0) as total_usage_minutes,
    avg(iff(monthly_usage_minutes > 0,monthly_usage_minutes,null)) as average_usage_minutes_per_month,
    median(iff(monthly_usage_minutes > 0,monthly_usage_minutes,null)) as median_usage_minutes_per_month,
    NULLIF(array_size(array_distinct(array_flatten(array_agg(weeks_aggregated)))),0) AS total_active_weeks,
    array_distinct(array_flatten(array_agg(weeks_aggregated))) as weeks_aggregated,
    MAX(HAS_PLACEMENT) as HAS_PLACEMENT,
    MAX(iff(monthly_usage_minutes > 0, max_login_date,null)) as max_login_date,
    MIN(iff(monthly_usage_minutes > 0, min_login_date,null)) as min_login_date
FROM monthly_usage
GROUP BY 1,2
), mra_monthly_skills as (
SELECT 
    'my_reading_academy' as AOFL_PRODUCT,
    USER_ID,
    DATE_TRUNC('month', ETL_TIMESTAMP) AS MONTH,
    SUM(IFF(SKILL_STATUS IN ('MASTERED', 'COMPLETED'), 1, 0)) AS COMPLETED_SKILLS,
    SUM(IFF(SKILL_STATUS = 'BYPASSED', 1, 0)) AS BYPASSED_SKILLS,
    SUM(IFF(SKILL_STATUS = 'IN_PROGRESS', 1, 0)) AS IN_PROGRESS_SKILLS,
    SUM(IFF(SKILL_STATUS = 'STRUGGLING', 1, 0)) AS STRUGGLING_SKILLS,
    SUM(SUM(IFF(SKILL_STATUS IN ('MASTERED', 'COMPLETED', 'BYPASSED'), 1, 0))) 
        OVER (PARTITION BY USER_ID) AS TOTAL_COMPLETED_BYPASSED,
    SUM(SUM(IFF(SKILL_STATUS IN ('MASTERED', 'COMPLETED'), 1, 0))) 
        OVER (PARTITION BY USER_ID) AS TOTAL_COMPLETED
    FROM mra_dw.dm.mra_user_skill_prog 
    -- WHERE ETL_TIMESTAMP BETWEEN '2023-07-01' AND DATEADD(day, -1, DATE_TRUNC('month', CURRENT_DATE())) // Defined in the scope of the project
    WHERE TO_DATE(ETL_TIMESTAMP) BETWEEN '2023-07-01' AND DATEADD(day, -1, CURRENT_DATE()) // Yesterday for QA
    GROUP BY 1,2,3
),
mma_monthly_skills as (
SELECT 
    'my_math_academy' AS AOFL_PRODUCT,
    USER_ID,
    DATE_TRUNC('month', ETL_TIMESTAMP) AS MONTH,
    SUM(IFF(SKILL_STATUS IN ('MASTERED', 'COMPLETED'), 1, 0)) AS COMPLETED_SKILLS,
    SUM(IFF(SKILL_STATUS = 'BYPASSED', 1, 0)) AS BYPASSED_SKILLS,
    SUM(IFF(SKILL_STATUS = 'IN_PROGRESS', 1, 0)) AS IN_PROGRESS_SKILLS,
    SUM(IFF(SKILL_STATUS = 'STRUGGLING', 1, 0)) AS STRUGGLING_SKILLS,
    SUM(SUM(IFF(SKILL_STATUS IN ('MASTERED', 'COMPLETED', 'BYPASSED'), 1, 0))) 
        OVER (PARTITION BY USER_ID) AS TOTAL_COMPLETED_BYPASSED,
    SUM(SUM(IFF(SKILL_STATUS IN ('MASTERED', 'COMPLETED'), 1, 0))) 
        OVER (PARTITION BY USER_ID) AS TOTAL_COMPLETED
FROM mma_dw.dm.user_skill_prog
-- WHERE ETL_TIMESTAMP BETWEEN '2023-07-01' AND DATEADD(day, -1, DATE_TRUNC('month', CURRENT_DATE())) // Defined in the scope of the project
WHERE TO_DATE(ETL_TIMESTAMP) BETWEEN '2023-07-01' AND DATEADD(day, -1, CURRENT_DATE()) // Yesterday for QA
GROUP BY 1,2,3
), total_skills as (
    SELECT a.*,
    b.* EXCLUDE (AOFL_PRODUCT,AOFL_PRODUCT_ID)
    FROM (SELECT * FROM mra_monthly_skills UNION ALL SELECT * FROM mma_monthly_skills) a
    LEFT JOIN (SELECT DISTINCT AOFL_PRODUCT, AOFL_PRODUCT_ID,STUDENT_ID,STUDENT_ROOT_ID,STUDENT_ROSTER_ID,STUDENT_EXTERNAL_ID 
                FROM EMS_DW.DW.EMS_DIM_STUDENTS WHERE IS_CURRENT = TRUE) b 
                on a.AOFL_PRODUCT = b.AOFL_PRODUCT AND a.USER_ID = b.AOFL_PRODUCT_ID
)

SELECT 
    a.* EXCLUDE (TEACHERS),
    SPLIT_PART(TEACHERS, '; ', 1) AS TEACHER1,
    SPLIT_PART(TEACHERS, '; ', 2) AS TEACHER2,
    SPLIT_PART(TEACHERS, '; ', 3) AS TEACHER3,
    CASE WHEN d.HAS_PLACEMENT = TRUE THEN 'YES'
         WHEN d.total_usage_minutes > 0 THEN 'IN PROGRESS'
         ELSE 'NO' END as PLACEMENT_STATUS,
    b.* EXCLUDE (AOFL_PRODUCT, STUDENT_ID,HAS_PLACEMENT,max_login_date,min_login_date, WEEKS_AGGREGATED),
    c.* EXCLUDE (AOFL_PRODUCT, STUDENT_ID,MONTH),
    d.* EXCLUDE (AOFL_PRODUCT, STUDENT_ID)
FROM base a
LEFT JOIN monthly_usage b USING (AOFL_PRODUCT,STUDENT_ID)
LEFT JOIN total_skills c USING (AOFL_PRODUCT,STUDENT_ID,MONTH)
LEFT JOIN total_usage d USING (AOFL_PRODUCT,STUDENT_ID)
WHERE ORGANIZATION_NAME = 'Tucson Unified School District'
-- AND b.MONTH BETWEEN '2023-07-01' AND DATEADD(day, -1, DATE_TRUNC('month', CURRENT_DATE())) // Defined in the scope of the project
AND b.MONTH BETWEEN '2023-07-01' AND DATEADD(day, -1, CURRENT_DATE()) // Yesterday for QA
-- AND a.STUDENT_ID = 1244188
ORDER BY a.AOFL_PRODUCT, a.STUDENT_ID, b.MONTH;
"""

# Define Parameters for the report
We need to tell the code where to point for all of the filenames in this report. You will need 3
1. Location of the PII data, provided by Tucson * 
2. Location of the usage data
3. Export Location

* Since this report isn't for all students, it is only for Tucson Native American students, we need a separate file to do the filtering. 
This file was provided by Tucson and has been manually edited to add in our internal ID. 

In [2]:
# Specify the path to your Excel file
file_path_dimensions = 'Student PII.xlsx' # Location of the PII data, provided by Tucson
file_path_usage = 'Tucson Full District Usage by Month 2024-03-26.csv' # Location of the usage data, just run and exported from Snowflake
file_path_export = 'Tucson Native American Student Usage - 2024-03-26.csv'

## Import data files
Using the file names provided above, import the data. 

In [3]:
import pandas as pd

# Use the read_excel function to read the file into a DataFrame
dtype_spec = { # Pull in the IDs as characters so they don't get corrupted. 
    'Perm ID' : 'str',
    'Internal ID' : 'str',
    'STUDENT_ID' : 'str'
}
df_pii = pd.read_excel(file_path_dimensions, engine='openpyxl',dtype=dtype_spec)
df_usage = pd.read_csv(file_path_usage,dtype=dtype_spec)

# Display the first few rows of the DataFrame to confirm successful import
# df_usage.head()


## Join the Usage and PII
Join the two files together, with the PII on the left to filter for only Tucson Native American students

In [4]:
df = pd.merge(left = df_pii,right = df_usage, left_on = ['Internal ID'],right_on=['STUDENT_ID'])
df['MONTHLY_SKILLS_MASTERED'] = df['COMPLETED_SKILLS'] + df['BYPASSED_SKILLS']
# df.head()

### Region join
Since I want to keep the School in our system, instead of the school from the original file, I need to make sure that I have the correct region tied to every individual student. So, first, I will create a lookup table for the regions, and then join that back in so that the SCHOOL_NAME from Snowflake has the correct region. 

In [5]:
# Region lookup
df_region_lookup = df_pii[['Region','School']].drop_duplicates().rename(columns={'Region':'REGION_NAME','School':'SCHOOL_NAME'})

df = pd.merge(left=df,right=df_region_lookup, on=['SCHOOL_NAME'])
# df
# df_region_lookup

,REGION_NAME,SCHOOL_NAME
0,Silverbell,Johnson Primary
6,Silverbell,Lawrence 3-8
7,Silverbell,Miller Elementary
14,Santa Cruz,Rose K-8
26,Santa Cruz,Roskruge Bilingual K-8 Magnet
29,Silverbell,White Elementary
38,Santa Cruz,Borton Magnet Elementary
41,Santa Cruz,Grijalva Elementary
51,Arroyo Chico,Lineweaver Elementary
54,Silverbell,Manzo Elementary


## Data Validation 1
1. Check for Duplicates from the join
2. Ensure that the school names match up
3. Check for where the Tucson Native American students are missing usage. 

In [6]:
# Check for Duplicate Instances of Perm ID
import pandas as pd

# Assuming df is your DataFrame
duplicates_df = df[df.duplicated(['AOFL_PRODUCT','Perm ID','MONTH'], keep=False)]  # keep=False to mark all duplicates as True
print(f"Total number of duplicates: {duplicates_df.shape[0]}")
duplicates_df.iloc[:, 0:13]


Total number of duplicates: 0


,Region,School,Perm ID,Last Name,First Name,Name,Grade,Gender,Internal ID,Notes,AOFL_PRODUCT,STUDENT_ID,ORGANIZATION_ID


In [7]:
import pandas as pd

# Dictionary of exceptions with STUDENT_ID as keys and a description as values
# These are students were rostering changes have been validated. 
exception_dict = {
    '1063923': 'Rostering changed from Lawrence to Dietz in January, still shown in PII file as Lawrence',
    '1063924': 'Rostering changed from Miller to Johnson in January',
    '1108589': 'Rostering changed from Johnson to Virtual Academy in September, still shown in PII file as Johnson',
    '1108673': 'Rostering changed from Roskruge to Mysers/Ganoung in November',
    '1459436': 'Rostering changed from Vesey to Banks in November, still shown in PII file as Vesey',
    '1459441': 'Rostering changed from Lawrence to Pueblo Gardens in November',
    '1505434': 'Rostering changed from Morgan to Tucson virtual in October, still shown in PII file as Morgan',
    '1543160': 'Rostering changed from Roskruge to Bienman virtual in January, still shown in PII file as Roskruge'
}

# Define a mask for rows where 'School' does not match 'SCHOOL_NAME'
mismatch_mask = df['School'].str.strip() != df['SCHOOL_NAME'].str.strip()

# Filter out the rows where STUDENT_ID is in the list of keys from the exception_dict
filtered_mismatch_mask = mismatch_mask & (~df['STUDENT_ID'].isin(exception_dict.keys()))

# Count mismatches after filtering out exceptions
mismatches_count = filtered_mismatch_mask.sum()

# Calculate the percentage of mismatches after filtering out exceptions
total_rows = df.shape[0]
mismatches_percentage = (mismatches_count / total_rows) * 100

# Filter rows where 'School' and 'SCHOOL_NAME' do not match, excluding exceptions
mismatches_df = df[filtered_mismatch_mask]

# Print message including the percentage of affected rows
print(f"Number of rows where 'School' does not match 'SCHOOL_NAME', excluding known exceptions: {mismatches_count} ({mismatches_percentage:.2f}%)")

# Optionally, print specific columns of mismatches_df to inspect these cases
mismatches_df[['Region', 'School', 'Perm ID', 'Name', 'STUDENT_ID', 'AOFL_PRODUCT', 'MONTH', 'SCHOOL_NAME']]


Number of rows where 'School' does not match 'SCHOOL_NAME', excluding known exceptions: 0 (0.00%)


,Region,School,Perm ID,Name,STUDENT_ID,AOFL_PRODUCT,MONTH,SCHOOL_NAME


In [8]:
import pandas as pd

# Assuming df is your pandas DataFrame
# Identify rows where 'TOTAL_USAGE_MINUTES' is zero or null
zero_or_null_minutes_mask = df['TOTAL_USAGE_MINUTES'].isnull() | (df['TOTAL_USAGE_MINUTES'] == 0)

# Count such rows
zero_or_null_count = zero_or_null_minutes_mask.sum()

# Filter rows meeting the condition
zero_or_null_df = df[zero_or_null_minutes_mask]

# Count the number of unique affected students
unique_affected_students = zero_or_null_df['STUDENT_ID'].nunique()

# Count the total number of unique students in the original DataFrame
total_unique_students = df['STUDENT_ID'].nunique()

# Calculate the percentage of unique affected students out of total unique students
percentage_affected_students = (unique_affected_students / total_unique_students) * 100

# Calculate the percentage of affected rows out of total rows
total_rows = df.shape[0]
percentage_affected_rows = (zero_or_null_count / total_rows) * 100

# Print messages including the counts and percentages
print(f"Number of affected rows with 'TOTAL_USAGE_MINUTES' being zero or null: {zero_or_null_count} out of {total_rows} total rows ({percentage_affected_rows:.2f}%)")
print(f"Number of unique affected students: {unique_affected_students} out of {total_unique_students} total students ({percentage_affected_students:.2f}%)")

# Optionally, print specific columns of zero_or_null_df to inspect these cases
# zero_or_null_df[['Region', 'School', 'Perm ID', 'Name', 'STUDENT_ID', 'AOFL_PRODUCT', 'MONTH', 'SCHOOL_NAME', 'TOTAL_USAGE_MINUTES']]


Number of affected rows with 'TOTAL_USAGE_MINUTES' being zero or null: 141 out of 3290 total rows (4.29%)
Number of unique affected students: 23 out of 215 total students (10.70%)


If the above data checks look good, then you can proceed with the export. 

## Pivot the data from long to wide, with columns for every month. 
The customer has requested that the data be in a format with the months as columns instead of rows. Meaning that we need to perform a pivot operation. However, before we are ready to do that, we need to handle an edge case. The edge case is when a student has usage in a given month, but not skills. That means that all of the dimensions related to skills are inaccurate for that month. To fix that, I created this compressed_df section where I compress all of the rows across the months for a student so that the dimensions would match up whether or not they had usage and skills in a given month. 


- A note here on which skills were chosen. The CS rep wanted all of the numbers to match educator center and within the report, meaning that we removed the bypassed skills and only reported on skills completed, using the same definition that educator center uses. 

In [9]:
# Create a clean list of the dimensions needed for the final dataset. 
# Correctly renaming only the week columns, preserving key columns for the merge
key_columns = ['REGION_NAME','SCHOOL_NAME','CLASSROOM_NAME', 'STUDENT_ID','USER_ID','Perm ID','Last Name', 'First Name ', 'Name',
               'Grade', 'Gender','AOFL_PRODUCT', 'PLACEMENT_STATUS',
               'TOTAL_ACTIVE_WEEKS','TOTAL_USAGE_MINUTES','TOTAL_COMPLETED',
               'MIN_LOGIN_DATE','MAX_LOGIN_DATE'
               ]
# print(df[key_columns].dtypes)
# Identify string and numeric columns
string_columns = [col for col in key_columns if df[col].dtype == object]
numeric_columns = [col for col in key_columns if col not in string_columns + ['STUDENT_ID', 'AOFL_PRODUCT']]

# Aggregate string columns by distinct LISTAGG
agg_string = {
    col: lambda x: '; '.join(sorted(set([str(val) for val in x if pd.notnull(val)])))
    for col in string_columns
}

# Aggregate numeric columns by average
agg_numeric = {col: 'max' for col in numeric_columns}

# Combine both aggregation dictionaries
aggregations = {**agg_string, **agg_numeric}

# Group by STUDENT_ID and AOFL_PRODUCT, then aggregate
compressed_df = df.groupby(['STUDENT_ID', 'AOFL_PRODUCT'], as_index=False).agg(aggregations)

In [10]:
# Pivot for weekly usage minutes using simplified key columns
pivot_usage = df.pivot_table(index=['STUDENT_ID', 'AOFL_PRODUCT'],
                             columns='MONTH', 
                             values='MONTHLY_USAGE_MINUTES',
                             aggfunc='sum',
                             fill_value=0).reset_index()

# Adjust column names for usage
pivot_usage.columns = [f'Minutes_{pd.to_datetime(col).strftime("%b%Y")}' if col not in ['STUDENT_ID', 'AOFL_PRODUCT'] and '-' in str(col) else col for col in pivot_usage.columns]

# Pivot for skills data
pivot_skills = df.pivot_table(index=['STUDENT_ID', 'AOFL_PRODUCT'],
                              columns='MONTH', 
                              values='COMPLETED_SKILLS', # Use the column that does not contain bypassed skills
                              aggfunc='sum',
                              fill_value=0).reset_index()

# Adjust column names for skills
pivot_skills.columns = [f'SkillsCompleted_{pd.to_datetime(col).strftime("%b%Y")}' if col not in ['STUDENT_ID', 'AOFL_PRODUCT'] and '-' in str(col) else col for col in pivot_skills.columns]
# Merge the pivoted usage and skills tables on STUDENT_ID and AOFL_PRODUCT
pivot_merged = pd.merge(pivot_usage, pivot_skills, 
                        on=['STUDENT_ID', 'AOFL_PRODUCT'], 
                        how='outer')

# # Join the other dimensions back
wide_df = pd.merge(compressed_df,pivot_merged, 
                   on=['STUDENT_ID', 'AOFL_PRODUCT'], 
                   how='left')

# # Display the first few rows to verify the transformation
# wide_df.head()

## Data Validation 2
1. Check for Duplicates after the pivot
2. Check for where the monthly skills don't match the total skills
3. Check for where the monthly usage doesn't match the total usage

In [11]:
# Check for Duplicate Instances of Perm ID
import pandas as pd

# Assuming df is your DataFrame
duplicates_df = wide_df[wide_df.duplicated(['AOFL_PRODUCT','Perm ID'], keep=False)]  # keep=False to mark all duplicates as True
print(f"Total number of duplicates: {duplicates_df.shape[0]}")
duplicates_df.iloc[:, 0:13]

Total number of duplicates: 0


,REGION_NAME,SCHOOL_NAME,CLASSROOM_NAME,STUDENT_ID,USER_ID,Perm ID,Last Name,First Name,Name,Grade,Gender,AOFL_PRODUCT,PLACEMENT_STATUS


In [12]:
# Step 1: Identify all SkillsMastered columns
skills_mastered_columns = [col for col in wide_df.columns if 'SkillsCompleted_' in col]

# Create a new DataFrame for testing to avoid modifying wide_df
test_df = wide_df.copy()
test_df['TOTAL_COMPLETED'] = test_df['TOTAL_COMPLETED'].fillna(0) # Missing should be zero for this check. 

# Add a column for the sum of SkillsMastered columns
test_df['SkillsCompleted_Sum'] = test_df[skills_mastered_columns].sum(axis=1)

# Add a column to flag mismatches
test_df['Mismatch'] = test_df['SkillsCompleted_Sum'] != test_df['TOTAL_COMPLETED']

# Step 2: Count and report the number of mismatches
mismatch_count = test_df['Mismatch'].sum()
total_rows = len(test_df)
mismatch_percentage = (mismatch_count / total_rows) * 100

print(f"Mismatched rows: {mismatch_count} ({mismatch_percentage:.2f}%)")

# Step 3: Print off the rows where the error is detected, including specific columns for clarity
mismatched_rows = test_df[test_df['Mismatch']]
columns_of_interest = ['AOFL_PRODUCT', 'SCHOOL_NAME', 'CLASSROOM_NAME', 'Name', 'TOTAL_COMPLETED'] + skills_mastered_columns
mismatched_rows[columns_of_interest]

Mismatched rows: 0 (0.00%)


,AOFL_PRODUCT,SCHOOL_NAME,CLASSROOM_NAME,Name,TOTAL_COMPLETED,SkillsCompleted_Jul2023,SkillsCompleted_Aug2023,SkillsCompleted_Sep2023,SkillsCompleted_Oct2023,SkillsCompleted_Nov2023,SkillsCompleted_Dec2023,SkillsCompleted_Jan2024,SkillsCompleted_Feb2024,SkillsCompleted_Mar2024


In [13]:
import pandas as pd

# Assuming wide_df is already defined

# Create a new DataFrame for testing
test_df_minutes = wide_df.copy()
test_df_minutes['TOTAL_USAGE_MINUTES'] = test_df_minutes['TOTAL_USAGE_MINUTES'].fillna(0) # Missing should be zero for this check. 

# Step 1: Identify all Minutes_ columns
minutes_columns = [col for col in wide_df.columns if col.startswith('Minutes_')]

# Step 2: Sum these columns for each row, round the sum and TOTAL_USAGE_MINUTES to the third decimal place, then compare
precision = 3
test_df_minutes['Minutes_Sum'] = test_df_minutes[minutes_columns].sum(axis=1)
test_df_minutes['Minutes_Mismatch'] = test_df_minutes['Minutes_Sum'].round(precision) != test_df_minutes['TOTAL_USAGE_MINUTES'].round(precision)

# Step 3: Count and report the number of mismatches for minutes, considering the rounding
minutes_mismatch_count = test_df_minutes['Minutes_Mismatch'].sum()
total_rows = len(test_df_minutes)
minutes_mismatch_percentage = (minutes_mismatch_count / total_rows) * 100

print(f"Minutes mismatched rows (considering rounding): {minutes_mismatch_count} ({minutes_mismatch_percentage:.2f}%)")

# Step 4: Print off the rows where the minutes error is detected, including the columns of interest
minutes_mismatched_rows = test_df_minutes[test_df_minutes['Minutes_Mismatch']]
columns_of_interest = ['STUDENT_ID','AOFL_PRODUCT', 'SCHOOL_NAME', 'CLASSROOM_NAME', 'Name', 'TOTAL_USAGE_MINUTES', 'Minutes_Sum'] + minutes_columns
minutes_mismatched_rows[columns_of_interest]


Minutes mismatched rows (considering rounding): 0 (0.00%)


,STUDENT_ID,AOFL_PRODUCT,SCHOOL_NAME,CLASSROOM_NAME,Name,TOTAL_USAGE_MINUTES,Minutes_Sum,Minutes_Jul2023,Minutes_Aug2023,Minutes_Sep2023,Minutes_Oct2023,Minutes_Nov2023,Minutes_Dec2023,Minutes_Jan2024,Minutes_Feb2024,Minutes_Mar2024


# File Export
Rename some of the dimensions from the file, clean up everything as it needs to be, and then export to CSV to be shared, via slack, with Sara Bleckinger. 

In [18]:
# Define the renaming dictionary
rename_dict = {
    'REGION_NAME' : 'Region Name',
    'SCHOOL_NAME' : 'School Name',
    'CLASSROOM_NAME' : 'Classroom Name',
    'AOFL_PRODUCT': 'AOFL Product',
    'PLACEMENT_STATUS': 'Placement Status',
    'TOTAL_USAGE_MINUTES': 'Total Minutes Year-to-Date',
    'TOTAL_ACTIVE_WEEKS': 'Active Weeks Year-to-Date',
    'TOTAL_COMPLETED_BYPASSED': 'Total Skills Mastered Year-to-Date',
    'TOTAL_COMPLETED' : 'Total Skills Completed Year-to-Date',
    'MIN_LOGIN_DATE' : 'First Login Date',
    'MAX_LOGIN_DATE' : 'Latest Login Date'
}

# Rename columns in df using the rename_dict
df_final = wide_df.rename(columns=rename_dict)
df_final = df_final.drop(columns=['STUDENT_ID','USER_ID'])

# Export df_final to a CSV file with the new filename
sort_columns = ['Region Name','School Name','Classroom Name','AOFL Product','Last Name','First Name ']
df_final.sort_values(by = sort_columns).to_csv(file_path_export, index=False)


# df_final.head()


# Deprecated Code

In [15]:
# # Query using the wrong skills table
# query = '''
# with base as (
#    -- Get rostering information based on what is active on the last day of the month
# SELECT
#     AOFL_PRODUCT,
#     STUDENT_ID,
#     ORGANIZATION_ID,
#     ORGANIZATION_NAME,
#     MONTH,
#     -- Concatenate together values from different active grades, schools, classrooms, and teachers
#     LISTAGG(DISTINCT GRADE, '; ') WITHIN GROUP (ORDER BY GRADE ASC) AS GRADE, 
#     LISTAGG(DISTINCT SCHOOL_NAME, '; ') WITHIN GROUP (ORDER BY SCHOOL_NAME ASC) AS SCHOOL_NAME, 
#     LISTAGG(DISTINCT CLASSROOM_NAME, '; ') WITHIN GROUP (ORDER BY CLASSROOM_NAME ASC) AS CLASSROOM_NAME,
#     LISTAGG(DISTINCT TEACHER_ID, '; ') WITHIN GROUP (ORDER BY TEACHER_ID ASC) AS teachers
# FROM SCHOOLS_DW.DM.MASTERY_LICENSED_ACTIVE_DAILY_SNAPSHOT
# WHERE ORGANIZATION_NAME = 'Tucson Unified School District'
#   AND DATE BETWEEN '2023-07-01' AND DATEADD(day, -1, DATE_TRUNC('month', CURRENT_DATE())) // Defined in the scope of the project
#   AND IS_LICENSED_SCHOOL = TRUE
#   AND IS_LICENSED_CLASSROOM = TRUE
#   AND STUDENT_GROUP_LINK_STATUS = 'active'
#   AND IS_LICENSED_STUDENT = TRUE
#   -- Teacher is not required, thus commented out
#   -- AND IS_LICENSED_TEACHER = TRUE
#   -- AND TEACHER_GROUP_STATUS = 'active'
#   AND DATE = LAST_DAY(DATE) -- Retrieves the last day of the month for the given DATE
# GROUP BY 1, 2, 3, 4, 5
# ), distinct_daily_usage as (
# // Since every line has multiple rows for a given day based on rostering information
# // I need to deduplicate the usage before aggregating. 
# SELECT DISTINCT
#     AOFL_PRODUCT,
#     STUDENT_ID,
#     DATE,
#     WEEK,
#     MONTH,
#     time_spend,
#     HAS_PLACEMENT
# FROM SCHOOLS_DW.DM.MASTERY_LICENSED_ACTIVE_DAILY_SNAPSHOT 
# WHERE ORGANIZATION_NAME = 'Tucson Unified School District'
# AND DATE BETWEEN '2023-07-01' AND DATEADD(day, -1, DATE_TRUNC('month', CURRENT_DATE())) // Defined in the scope of the project
# ), monthly_usage as (
#  SELECT 
#     AOFL_PRODUCT,
#     STUDENT_ID,
#     MONTH,
#     sum(time_spend) / 60 as monthly_usage_minutes,
#     MAX(HAS_PLACEMENT) as HAS_PLACEMENT,
#     MAX(iff(time_spend > 0, DATE,null)) as max_login_date,
#     MIN(iff(time_spend > 0, DATE,null)) as MIN_login_date,
#     ARRAY_DISTINCT(ARRAY_AGG(DISTINCT IFF(time_spend > 0, WEEK, NULL))) as weeks_aggregated
# FROM distinct_daily_usage
# GROUP BY 1,2,3
# ), total_usage as (
# SELECT 
#     AOFL_PRODUCT,
#     STUDENT_ID,
#     NULLIF(sum(monthly_usage_minutes), 0) as total_usage_minutes,
#     avg(iff(monthly_usage_minutes > 0,monthly_usage_minutes,null)) as average_usage_minutes_per_month,
#     median(iff(monthly_usage_minutes > 0,monthly_usage_minutes,null)) as median_usage_minutes_per_month,
#     NULLIF(array_size(array_distinct(array_flatten(array_agg(weeks_aggregated)))),0) AS total_active_weeks,
#     array_distinct(array_flatten(array_agg(weeks_aggregated))) as weeks_aggregated,
#     MAX(HAS_PLACEMENT) as HAS_PLACEMENT,
#     MAX(iff(monthly_usage_minutes > 0, max_login_date,null)) as max_login_date,
#     MIN(iff(monthly_usage_minutes > 0, min_login_date,null)) as min_login_date
# FROM monthly_usage
# GROUP BY 1,2
# ), monthly_skills as (
# SELECT 
#     AOFL_PRODUCT,
#     MONTH,
#     STUDENT_ID,
#     NULLIF(array_size(array_distinct(array_flatten(array_agg(mastered_skills)))),0) as monthly_skills_mastered,
#     array_distinct(array_flatten(array_agg(mastered_skills))) as mastered_skills
# FROM SCHOOLS_DW.DM.MASTERY_SKILL_DAILY_SNAPSHOT
# WHERE TERM_ID = 2
# AND DATE BETWEEN '2023-07-01' AND DATEADD(day, -1, DATE_TRUNC('month', CURRENT_DATE())) // Defined in the scope of the project
# GROUP BY 1,2,3
# ),
# total_skills as (
# SELECT 
#     AOFL_PRODUCT,
#     STUDENT_ID,
#     NULLIF(array_size(array_distinct(array_flatten(array_agg(mastered_skills)))),0) as total_skills_mastered,
#     avg(iff(monthly_skills_mastered > 0,monthly_skills_mastered,null)) as average_monthly_skills_per_month,
#     median(iff(monthly_skills_mastered > 0,monthly_skills_mastered,null)) as median_monthly_skills_per_month -- CONFIRM AVERAGE OR MEDIAN
# FROM monthly_skills
# GROUP BY 1,2
# )

# SELECT 
#     a.* EXCLUDE (TEACHERS),
#     SPLIT_PART(TEACHERS, '; ', 1) AS TEACHER1,
#     SPLIT_PART(TEACHERS, '; ', 2) AS TEACHER2,
#     SPLIT_PART(TEACHERS, '; ', 3) AS TEACHER3,
#     CASE WHEN d.HAS_PLACEMENT = TRUE THEN 'YES'
#          WHEN d.total_usage_minutes > 0 THEN 'IN PROGRESS'
#          ELSE 'NO' END as PLACEMENT_STATUS,
#     b.* EXCLUDE (AOFL_PRODUCT, STUDENT_ID,MONTH,HAS_PLACEMENT,max_login_date,min_login_date, WEEKS_AGGREGATED),
#     c.* EXCLUDE (AOFL_PRODUCT, STUDENT_ID,MONTH,MASTERED_SKILLS),
#     d.* EXCLUDE (AOFL_PRODUCT, STUDENT_ID),
#     e.* EXCLUDE (AOFL_PRODUCT, STUDENT_ID)
# FROM base a
# LEFT JOIN monthly_usage b USING (AOFL_PRODUCT,STUDENT_ID,MONTH)
# LEFT JOIN monthly_skills c USING (AOFL_PRODUCT,STUDENT_ID,MONTH)
# LEFT JOIN total_usage d USING (AOFL_PRODUCT,STUDENT_ID)
# LEFT JOIN total_skills e USING (AOFL_PRODUCT,STUDENT_ID)
# WHERE ORGANIZATION_NAME = 'Tucson Unified School District'
# AND a.MONTH BETWEEN '2023-07-01' AND DATEADD(day, -1, DATE_TRUNC('month', CURRENT_DATE())) // Defined in the scope of the project
# ORDER BY a.AOFL_PRODUCT, a.STUDENT_ID, a.MONTH;
# '''

In [16]:
# # older pivot code before the skills missing from month adjustment
# import pandas as pd
# import datetime

# # Correctly renaming only the week columns, preserving key columns for the merge
# key_columns = ['REGION_NAME','SCHOOL_NAME','CLASSROOM_NAME', 'STUDENT_ID','USER_ID','Perm ID','Last Name', 'First Name ', 'Name',
#                'Grade', 'Gender','AOFL_PRODUCT', 'PLACEMENT_STATUS',
#                'TOTAL_ACTIVE_WEEKS','TOTAL_USAGE_MINUTES','TOTAL_COMPLETED','TOTAL_COMPLETED_BYPASSED',
#                'MIN_LOGIN_DATE','MAX_LOGIN_DATE'
#                ]


# # Pivot for weekly usage minutes
# pivot_usage = df.pivot_table(index=key_columns,
#                              columns='MONTH', 
#                              values='MONTHLY_USAGE_MINUTES',
#                              aggfunc='sum',
#                              fill_value=0).reset_index()

# # pivot_usage.columns = [f'{col}_usage' if col not in key_columns and isinstance(col, str) else col for col in pivot_usage.columns]
# pivot_usage.columns = [f'Minutes_{pd.to_datetime(col).strftime('%b%Y')}' if col not in key_columns and '-' in str(col) else col for col in pivot_usage.columns]

# # Assuming you have skills data in the same DataFrame and a similar WEEK structure
# pivot_skills = df.pivot_table(index=key_columns,
#                               columns='MONTH', 
#                               values='COMPLETED_SKILLS',  # Adjust based on your actual column for weekly skills
#                               aggfunc='sum',  # Adjust this based on how you want to aggregate skills data
#                               fill_value=0).reset_index()

# # pivot_skills.columns = [f'{col}_skills' if col not in key_columns and isinstance(col, str) else col for col in pivot_skills.columns]
# pivot_skills.columns = [f'SkillsMastered_{pd.to_datetime(col).strftime('%b%Y')}' if col not in key_columns and '-' in str(col) else col for col in pivot_skills.columns]



# # Merge the pivoted usage and skills tables on common identifiers
# wide_df = pd.merge(pivot_usage, pivot_skills, 
#                    on=key_columns, 
#                    how='outer')

# # Print the first few rows to verify the transformation
# # wide_df.head()
# wide_df.to_clipboard()